In [2]:
pip install pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('PySparkApp').getOrCreate()

In [5]:
spark

In [6]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [7]:
pdf=ps.DataFrame(
    {'a':[1,2,3,4,5,6],
     'b':[100,200,300,400,500,600],
     'c': ["one","two","three","four","five","six"]
     },
    index=[1,2,3,4,5,6]

)
print(pdf)

   a    b      c
1  1  100    one
2  2  200    two
3  3  300  three
4  4  400   four
5  5  500   five
6  6  600    six


In [8]:
test=ps.DataFrame([("Jack","","Eldridge","36636","M",90000),
    ("Matthew","J", "Munro","28832","M",45400),
    ("Sheila","Oway", "Roberts","12114","F",64000),
    ("Anne","", "Dushane","32192","F",141000),
    ("Jane","Rebecca","Jones","99482","F",56000)])
print(test)
test.dtypes

         0        1         2      3  4       5
0     Jack           Eldridge  36636  M   90000
1  Matthew        J     Munro  28832  M   45400
2   Sheila     Oway   Roberts  12114  F   64000
3     Anne            Dushane  32192  F  141000
4     Jane  Rebecca     Jones  99482  F   56000


,0
0,object
1,object
2,object
3,object
4,object
5,int64


In [9]:
print(test.to_markdown())

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_markdown` loads all data into the driver's memory. It should only be used if the resulting pandas object is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


|    | 0       | 1       | 2        |     3 | 4   |      5 |
|---:|:--------|:--------|:---------|------:|:----|-------:|
|  0 | Jack    |         | Eldridge | 36636 | M   |  90000 |
|  1 | Matthew | J       | Munro    | 28832 | M   |  45400 |
|  2 | Sheila  | Oway    | Roberts  | 12114 | F   |  64000 |
|  3 | Anne    |         | Dushane  | 32192 | F   | 141000 |
|  4 | Jane    | Rebecca | Jones    | 99482 | F   |  56000 |


In [10]:
test.columns

Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [13]:
df=pd.read_csv("property-sales.csv")
df

,Address,Type,City,SalePrice ($),Agent
0,1 Rowley Street,Detached House,New York,745000,Penelope Pullman
1,13a lollipop avenue,Apartment,Los Angeles,345000,Jack Smith
2,34 the drive,House,Atlanta,459000,Sheila Sammi


In [14]:
# Declare the path to our file
csv_path = '/content/property-sales.csv'

# Read a csv file from Files/property-sales.csv
df_csv = spark.read.csv(csv_path, header=True)

display(df_csv)

DataFrame[Address : string, Type: string, City : string, SalePrice ($): string, Agent: string]

In [15]:
df_csv.show()

+-------------------+--------------+-----------+-------------+-----------------+
|           Address |          Type|      City |SalePrice ($)|            Agent|
+-------------------+--------------+-----------+-------------+-----------------+
|   1 Rowley Street |Detached House|   New York|       745000|Penelope Pullman |
|13a lollipop avenue|     Apartment|Los Angeles|       345000|      Jack Smith |
|       34 the drive|         House|    Atlanta|       459000|     Sheila Sammi|
+-------------------+--------------+-----------+-------------+-----------------+



In [16]:

df_csv.dtypes

[('Address ', 'string'),
 ('Type', 'string'),
 ('City ', 'string'),
 ('SalePrice ($)', 'string'),
 ('Agent', 'string')]

In [17]:
#Write the csv file to a json file
df_csv.write.json("Files/json/property-sales.json", mode='overwrite')

In [18]:

df_json = spark.read.json('Files/json/property-sales.json')
display(df_json)

DataFrame[Address : string, Agent: string, City : string, SalePrice ($): string, Type: string]

In [19]:
df_json.write.parquet('Files/parquet/property-sales2.parquet', mode='overwrite')

In [20]:
df_parquet = spark.read.parquet('Files/parquet/property-sales2.parquet')
display(df_parquet)
df_parquet.show()

DataFrame[Address : string, Agent: string, City : string, SalePrice ($): string, Type: string]

+-------------------+-----------------+-----------+-------------+--------------+
|           Address |            Agent|      City |SalePrice ($)|          Type|
+-------------------+-----------------+-----------+-------------+--------------+
|   1 Rowley Street |Penelope Pullman |   New York|       745000|Detached House|
|13a lollipop avenue|      Jack Smith |Los Angeles|       345000|     Apartment|
|       34 the drive|     Sheila Sammi|    Atlanta|       459000|         House|
+-------------------+-----------------+-----------+-------------+--------------+



In [21]:
# read all the parquet files in the 'Files/parquet/' folder into a dataframe
df_all_parquet = spark.read.parquet('Files/parquet/*.parquet')

In [22]:
df = spark.read.json('Files/json/property-sales.json')
display(df)

DataFrame[Address : string, Agent: string, City : string, SalePrice ($): string, Type: string]

In [23]:
df.printSchema()

root
 |-- Address : string (nullable = true)
 |-- Agent: string (nullable = true)
 |-- City : string (nullable = true)
 |-- SalePrice ($): string (nullable = true)
 |-- Type: string (nullable = true)



In [24]:
df = df.withColumnRenamed("SalePrice ($)","SalePrice_USD")\
        .withColumnRenamed("Address ", "Address")\
        .withColumnRenamed("City ", "City")

In [25]:
df.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Agent: string (nullable = true)
 |-- City: string (nullable = true)
 |-- SalePrice_USD: string (nullable = true)
 |-- Type: string (nullable = true)



In [27]:
df.show()

+-------------------+-----------------+-----------+-------------+--------------+
|            Address|            Agent|       City|SalePrice_USD|          Type|
+-------------------+-----------------+-----------+-------------+--------------+
|   1 Rowley Street |Penelope Pullman |   New York|       745000|Detached House|
|13a lollipop avenue|      Jack Smith |Los Angeles|       345000|     Apartment|
|       34 the drive|     Sheila Sammi|    Atlanta|       459000|         House|
+-------------------+-----------------+-----------+-------------+--------------+



In [26]:
csv_path='/content/property-sales.csv'
df_pandas=ps.read_csv(csv_path)
df_pandas

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,Address,Type,City,SalePrice ($),Agent
0,1 Rowley Street,Detached House,New York,745000,Penelope Pullman
1,13a lollipop avenue,Apartment,Los Angeles,345000,Jack Smith
2,34 the drive,House,Atlanta,459000,Sheila Sammi


In [28]:
df_convert=df_pandas.to_spark()
df_convert.show()

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+-------------------+--------------+-----------+-------------+-----------------+
|           Address |          Type|      City |SalePrice ($)|            Agent|
+-------------------+--------------+-----------+-------------+-----------------+
|   1 Rowley Street |Detached House|   New York|       745000|Penelope Pullman |
|13a lollipop avenue|     Apartment|Los Angeles|       345000|      Jack Smith |
|       34 the drive|         House|    Atlanta|       459000|     Sheila Sammi|
+-------------------+--------------+-----------+-------------+-----------------+



In [29]:
import json
# Example: Convert Spark DataFrame to JSON
df_csv.write.json("property-sales_spark.json", mode='overwrite') # Writing with Spark
df_json=spark.read.json("property-sales_spark.json")
pandas_df=df_json.toPandas()
pandas_df.to_json("property-sales_pandas.json") # Writing with Pandas

print("JSON file created successfully")


JSON file created successfully


In [30]:
df = spark.read.json('Files/json/property-sales.json')
df.printSchema()

root
 |-- Address : string (nullable = true)
 |-- Agent: string (nullable = true)
 |-- City : string (nullable = true)
 |-- SalePrice ($): string (nullable = true)
 |-- Type: string (nullable = true)



In [31]:
# changing column names to allow write to Lakehouse tables
df = df.withColumnRenamed("SalePrice ($)","SalePrice_USD")\
        .withColumnRenamed("Address ", "Address")\
        .withColumnRenamed("City ", "City")
df.printSchema()


root
 |-- Address: string (nullable = true)
 |-- Agent: string (nullable = true)
 |-- City: string (nullable = true)
 |-- SalePrice_USD: string (nullable = true)
 |-- Type: string (nullable = true)



In [32]:
import pyspark.pandas as ps
psdf=df.to_pandas_on_spark()
psdf

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:5725: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


,Address,Agent,City,SalePrice_USD,Type
0,1 Rowley Street,Penelope Pullman,New York,745000,Detached House
1,13a lollipop avenue,Jack Smith,Los Angeles,345000,Apartment
2,34 the drive,Sheila Sammi,Atlanta,459000,House


In [33]:
# First we need a DataFrame to look at!
df = spark.read.csv('/content/property-sales.csv', header=True, inferSchema=True)


In [34]:
df.show()
display(df)

+-------------------+--------------+-----------+-------------+-----------------+
|           Address |          Type|      City |SalePrice ($)|            Agent|
+-------------------+--------------+-----------+-------------+-----------------+
|   1 Rowley Street |Detached House|   New York|       745000|Penelope Pullman |
|13a lollipop avenue|     Apartment|Los Angeles|       345000|      Jack Smith |
|       34 the drive|         House|    Atlanta|       459000|     Sheila Sammi|
+-------------------+--------------+-----------+-------------+-----------------+



DataFrame[Address : string, Type: string, City : string, SalePrice ($): int, Agent: string]

In [35]:
display(df.head(2))

[Row(Address ='1 Rowley Street ', Type='Detached House', City ='New York', SalePrice ($)=745000, Agent='Penelope Pullman '),
 Row(Address ='13a lollipop avenue', Type='Apartment', City ='Los Angeles', SalePrice ($)=345000, Agent='Jack Smith ')]

In [36]:
df.printSchema()
df.dtypes

root
 |-- Address : string (nullable = true)
 |-- Type: string (nullable = true)
 |-- City : string (nullable = true)
 |-- SalePrice ($): integer (nullable = true)
 |-- Agent: string (nullable = true)



[('Address ', 'string'),
 ('Type', 'string'),
 ('City ', 'string'),
 ('SalePrice ($)', 'int'),
 ('Agent', 'string')]

In [37]:
df.schema

StructType([StructField('Address ', StringType(), True), StructField('Type', StringType(), True), StructField('City ', StringType(), True), StructField('SalePrice ($)', IntegerType(), True), StructField('Agent', StringType(), True)])

In [38]:
source_schema=df.schema
##Storing schema as source schema
from os import path
# this saves us having to explicitly write out our the schema for a new df, if we have one that already exists.
new_df_with_existing_schema = spark.read.csv( schema=source_schema,path='/content/property-sales.csv', header=True)


In [39]:
df.columns

['Address ', 'Type', 'City ', 'SalePrice ($)', 'Agent']

In [40]:
df.select('Type').show()

+--------------+
|          Type|
+--------------+
|Detached House|
|     Apartment|
|         House|
+--------------+



In [41]:
#renaming existing columns
df = df.withColumnRenamed('Address ', 'Address')
df.select('Address').show()

+-------------------+
|            Address|
+-------------------+
|   1 Rowley Street |
|13a lollipop avenue|
|       34 the drive|
+-------------------+



In [42]:
# selecting a few columns
df.select(['Address','Type']).show()

+-------------------+--------------+
|            Address|          Type|
+-------------------+--------------+
|   1 Rowley Street |Detached House|
|13a lollipop avenue|     Apartment|
|       34 the drive|         House|
+-------------------+--------------+



In [ ]:
df = df.withColumn('2x_SalePrice', df['SalePrice ($)'] * 2)
df.show()


In [ ]:
df = df.drop('2x_SalePrice')
df.show()


In [ ]:
df.show()


In [ ]:
df_new = df.selectExpr("Address as ADD","'SalePrice ($)' as SalesPrice_USD","'City ' as MyCity")
df_new.show()